In [0]:
from dataclasses import dataclass
from itertools import chain
from typing import Union
from pyspark.ml.feature import Bucketizer

In [0]:
%run ./utils

In [0]:
%run ./_setup

In [0]:
# Set constants
COLS_TO_DROP = [
    "_c0",
    "CompTotal",
    "CompFreq",
    "SurveyLength",
    "SurveyEase",
]
COLS_TO_SPLIT = [
    "DevType",
    "LanguageHaveWorkedWith",
    "LanguageWantToWorkWith",
    "DatabaseHaveWorkedWith",
    "DatabaseWantToWorkWith",
    "PlatformHaveWorkedWith",
    "PlatformWantToWorkWith",
    "WebframeHaveWorkedWith",
    "WebframeWantToWorkWith",
    "MiscTechHaveWorkedWith",
    "MiscTechWantToWorkWith",
    "ToolsTechHaveWorkedWith",
    "ToolsTechWantToWorkWith",
    "NEWCollabToolsHaveWorkedWith",
    "NEWCollabToolsWantToWorkWith",
    "NEWStuck",
]
FINAL_DROP = list(chain(COLS_TO_DROP, COLS_TO_SPLIT))

Out[32]: {'EdLevel': {'Primary/elementary school': 1.0,
  'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 2.0,
  'Associate degree (A.A., A.S., etc.)': 3.0,
  'Some college/university study without earning a degree': 4.0,
  'Something else, Professional degree (JD, MD, etc.)': 5.0,
  'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 6.0,
  'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 7.0,
  'Other doctoral degree (Ph.D., Ed.D., etc.)': 8.0},
 'Age1stCode': {'Younger than 5 years': 1.0,
  '5 - 10 years': 2.0,
  '11 - 17 years': 3.0,
  '18 - 24 years': 4.0,
  '25 - 34 years': 5.0,
  '35 - 44 years': 6.0,
  '45 - 54 years': 7.0,
  '55 - 64 years': 8.0,
  'Older than 64 years': 9.0},
 'OrgSize': {'Just me - I am a freelancer, sole proprietor, etc.': 1.0,
  '2 to 9 employees': 2.0,
  '10 to 19 employees': 3.0,
  '20 to 99 employees': 4.0,
  '100 to 499 employees': 5.0,
  'I don’t know': 6.0,
  '500 to 999 employees': 7.0,
  '1,000 to 4,999 employees

In [0]:
def group_low_freq(
    *, df: DataFrame, col_name: str, threshold: float = 1.0, group_name: str = "Other"
) -> DataFrame:
    """
    Group all observations that occur in less then a threshold % of the rows in df column.
    :param df: A pyspark.sql.dataframe.DataFrame
    :param col_name: The relevant column to run the function on.
    :param threshold: Values below the threshold will be set as group_name or 'Other' category.
    :param group_name: String to use as replacement for the observations that need to be grouped.
    :return df: A pyspark.sql.dataframe.DataFrame object.
    """
    col_df = count_by_col(df=df, col_name=col_name)
    low_freq = col_df.filter(col_df["Percentage (%)"] < threshold)
    others = list(map(lambda row: row[0], low_freq.select(col_name).collect()))
    df_with_other = df.withColumn(
        col_name, f.when(df[col_name].isin(others), "Other").otherwise(df[col_name])
    )

    return df_with_other

In [0]:
def group_low_freq_cols(
    *, df: DataFrame, cols_to_group: List[str] = ["Country", "Currency", "Ethnicity"]
) -> DataFrame:
    """The function gets a list of columns as an input and set categories with low freaquency to 'Other' category.
    :param df: A pyspark.sql.dataframe.DataFrame object.
    :param cols_to_group: A list of columns names.
    :return df: A pyspark.sql.dataframe.DataFrame object.
    """
    for col_name in cols_to_group:
        df = group_low_freq(df=df, col_name=col_name)

    return df

In [0]:
def convert_years_cols(
    *, df: DataFrame, years_cols: List[str] = ["YearsCode", "YearsCodePro"]
) -> DataFrame:
    """The function converts the years column from string type to int type.
    :param df: A pyspark.sql.dataframe.DataFrame object.
    :param years_cols: A list of columns names.
    :return df: A pyspark.sql.dataframe.DataFrame object.
    """
    for col_name in years_cols:
        df = df.withColumn(
            col_name,
            f.when(f.col(col_name) == "Less than 1 year", 0)
            .when(f.col(col_name) == "More than 50 years", 51)
            .when(f.col(col_name) == "NA", 999)
            .otherwise(f.col(col_name))
            .cast("integer"),
        )

    return df

In [0]:
def set_bins(
    *, df: DataFrame, splits: List[Union[int, float]], inputCol: str
) -> DataFrame:
    """
    Sets bins for df column values.
    :param df: A pyspark.sql.dataframe.DataFrame
    :param splits: A list of bins to split by.
    :param inputCol: The column name that needs to be splitted to bins.
    :param outputCol: The name of the output column.
    :return df: A pyspark.sql.dataframe.DataFrame object.
    """
    bucketizer = Bucketizer(
        splits=splits, inputCol=inputCol, outputCol=f"{inputCol}_rank"
    )
    buck_df = bucketizer.transform(df)
    # Format bins categories
    format_udf = udf(
        lambda x: f"{splits[int(x)]} - {(splits[int(x) + 1])}", t.StringType()
    )
    bins_df = (
        buck_df.withColumn(f"{inputCol}_bin", format_udf(f"{inputCol}_rank"))
        .drop(inputCol)
        .drop(f"{inputCol}_bin")
    )

    return bins_df

In [0]:
def set_bins_for_years_cols(
    *,
    df: DataFrame,
    years_bins: List[int] = [0, 3, 5, 10, 20, 30, 40, 50, 60, 1000],
    cols_to_bins: List[str] = ["YearsCode", "YearsCodePro"]
) -> DataFrame:
    """
    Sets bins for years columns values of the DataFrame.
    :param df: A pyspark.sql.dataframe.DataFrame
    :param years_bins: A list of bins to split by.
    :param cols_to_bins: The columns names that needs to be splitted to bins.
    :return df: A pyspark.sql.dataframe.DataFrame
    """
    for col_name in cols_to_bins:
        df = set_bins(df=df, splits=years_bins, inputCol=col_name)

    return df

In [0]:
def replace_to_null(
    *,
    df: DataFrame,
    cols: List[str] = ["YearsCode_rank", "YearsCodePro_rank"],
    value: Union[str, int] = 8
) -> DataFrame:
    """The function gets a DataFrame, col name, and a value and returns a modified DataFrame with None values isntead of the input value.
    :param df: A pyspark.sql.dataframe.DataFrame
    :param cols: A list of columns names.
    :return df: A pyspark.sql.dataframe.DataFrame object.
    """
    for col_name in cols:
        df = df.withColumn(
            col_name,
            f.when(f.col(col_name) != value, f.col(col_name)).otherwise(f.lit(None)),
        )
    return df

In [0]:
def map_an_ordinal_col(
    *, df: DataFrame, col_name: str, mapping: Dict[str, int]
) -> DataFrame:
    mapping_expr = f.create_map([f.lit(x) for x in chain(*mapping.items())])
    df = df.withColumn(f"{col_name}_rank", mapping_expr.getItem(f.col(col_name))).drop(
        col_name
    )
    return df

In [0]:
def map_ordinal_cols(
    *, df: DataFrame, mapping_dict: Dict[str, Dict[str, int]]
) -> DataFrame:
    for col_name, mapping in mapping_dict.items():
        df = map_an_ordinal_col(df=df, col_name=col_name, mapping=mapping)
    return df

In [0]:
# The function below splits a category type column to many columns of the categories with the values of 1 or 0 in each.
# So why not just using built in OneHotEncoder method of sklearn.preprocessing module?
# Because the categorical columns in this dataset incloudes thousends of combinations of categorical variables with A LOT of low-represented class Using OneHotEncoder directly, will create a lot of variables.
def split_a_column(
    *, df: DataFrame, col_name: str, delimiter: str = ";", prefix: str = "_"
) -> DataFrame:
    """The function splits a category type column to many columns of the categories with the values of 1 or 0 in each.
    :param df: A pyspark.sql.dataframe.DataFrame
    :param col_name: The column that needs to be splitted.
    :param delimiter: The delimiter sign. Ex: '|', ';'.
    :param prefix: The new splitted columns prefix. Set to default of _ as prefix.
    :return df: A pyspark.sql.dataframe.DataFrame object.
    """
    delim_df = df.withColumn(f"{col_name}Arr", f.split(col_name, delimiter))
    all_items = delim_df.select(f"{col_name}Arr").rdd.flatMap(lambda x: x).collect()
    flat_list = [
        item for sub_list in all_items for item in sub_list if str(item) != "NA"
    ]
    unique_items = set(flat_list)
    for item in unique_items:
        item = item.replace(".", " ")
        new_col_name = f"{col_name}{prefix}{item}"
        df = df.withColumn(
            f"{new_col_name}", f.when(f.col(col_name).like(f"%{item}%"), 1).otherwise(0)
        )

    return df.drop(col_name)

In [0]:
def split_multiple_columns(*, df: DataFrame, cols: List[str]) -> DataFrame:
    """The function gets a list of columns names and runs the function split_column on the columns in a loop.
    :param df: A pyspark.sql.dataframe.DataFrame
    :param cols: A list of columns to run the function on.
    :param prefix: The new splitted columns prefix. Set to default of no prefix.
    :return df: A pyspark.sql.dataframe.DataFrame object.
    """
    for count, col_name in enumerate(cols):
        df = split_a_column(df=df, col_name=col_name)

    return df

In [0]:
def preprocess_pipeline(*, df: DataFrame) -> DataFrame:
    """
    :param df: A pyspark.sql.dataframe.DataFrame
    :return step_7: A pyspark.sql.dataframe.DataFrame object.
    """
    step_1 = group_low_freq_cols(df=df)
    step_2 = floor_and_cap_outliers(df=step_1)
    step_3 = convert_years_cols(df=step_2)
    step_4 = set_bins_for_years_cols(df=step_3)
    step_5 = replace_to_null(df=step_4)
    step_6 = map_ordinal_cols(df=step_5, mapping_dict=ORDINAL_MAPPING)
    print(f"Split the following columns: \n {COLS_TO_SPLIT}")
    step_7 = split_multiple_columns(df=step_6, cols=COLS_TO_SPLIT)
    step_8 = step_7.drop(*FINAL_DROP)
    step_9 = save_table(
        df=step_8, file_path=f"s3a://{S3_PROCESS_PATH}processed_survey.parquet"
    )
    return None